In [30]:
from datasets import load_dataset
dataset = load_dataset("allenai/tulu-v2-sft-mixture")
dataset

DatasetDict({
    train: Dataset({
        features: ['dataset', 'id', 'messages'],
        num_rows: 326154
    })
})

In [31]:
# judge the prompt or response is empty
def flag_empty(e):
    e["empty"] = False
    for m in e["messages"]:
        if m["content"].strip() == "" and m["role"] != "system":
            e["empty"] = True
    return e

_dataset = dataset.map(flag_empty, num_proc=8)
_empty_ds = _dataset.filter(lambda x: x["empty"], num_proc=8)
_dataset_notempty = _dataset.filter(lambda x: not x["empty"], num_proc=8)

In [32]:
_empty_ds

DatasetDict({
    train: Dataset({
        features: ['dataset', 'id', 'messages', 'empty'],
        num_rows: 270
    })
})

In [27]:
export_dataset

DatasetDict({
    train: Dataset({
        features: ['dataset', 'id', 'messages'],
        num_rows: 325884
    })
})

In [33]:
# export to json
export_dataset = _dataset_notempty.select_columns(["dataset", "id", "messages"])
export_dataset["train"].to_json("./tulu_v2.json",num_proc=8)

Creating json from Arrow format:  97%|█████████▋| 317/326 [00:03<00:00, 125.48ba/s]

Creating json from Arrow format: 100%|██████████| 326/326 [00:03<00:00, 94.95ba/s] 


1414664165

In [71]:
# role type
def get_role(e):
    e['role'] = [x['role'] for x in e['messages']]
    return e
data_role = dataset.map(get_role, num_proc=8)
role_list = data_role['train']['role']
roles = [item for sublist in role_list for item in sublist]
set(roles)

{'assistant', 'system', 'user'}

In [49]:
len(dataset['train'][0]['messages'])
def data_length(e):
    e['length'] = len(e["messages"])
    return e
dataset_length = dataset.map(data_length, num_proc=8)
dataset_length
# mutil turns dataset
dataset_mutil_turn = dataset_length.filter(lambda x: x['length'] > 3, num_proc=8)
dataset_mutil_turn
set(dataset_mutil_turn['train']['dataset'])

Filter (num_proc=8):   0%|          | 0/326154 [00:00<?, ? examples/s]

Filter (num_proc=8): 100%|██████████| 326154/326154 [00:02<00:00, 123977.67 examples/s]


DatasetDict({
    train: Dataset({
        features: ['dataset', 'id', 'messages', 'length'],
        num_rows: 29683
    })
})

In [72]:
#dialogue turns count
from collections import Counter  
data_length = dataset_length['train']['length']
counter = Counter(data_length) 
# for value, count in counter.items():  
#     print(f"Value: {value}, Count: {count}")  
import pandas as pd
data_length_distribution = pd.Series(counter)
data_length_distribution.sort_index(inplace=True)
data_length_distribution.to_csv("./turns_distribution.csv")